# **Vehicle Detection** 
## Overview 
Develop a pipeline to identify the lane boundaries from a front-facing camera on a car. Extensive use of OpenCV, matplotlib, numpy will be used.

The goals / steps of this project are the following:
- Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier
- Optionally, you can also apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector. 
- Note: for those first two steps don't forget to normalize your features and randomize a selection for training and testing.
- Implement a sliding-window technique and use your trained classifier to search for vehicles in images.
- Run your pipeline on a video stream (start with the test_video.mp4 and later implement on full project_video.mp4) and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
- Estimate a bounding box for vehicles detected.

## Import Packages

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import cv2
import glob
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib

%matplotlib inline

## SDC Helper Functions

In [ ]:
# Feature extraction parameters
color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
spatial_size = (32, 32)
hist_bins = 32
orient = 11
pix_per_cell = 16
cell_per_block = 2
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"

def add_heat( heatmap, bbox_list ):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap

def apply_threshold( heatmap, threshold ):
    # Zero out pixels below the threshold
    heatmap[ heatmap <= threshold ] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes( img, labels ):
    # Iterate through all detected cars
    for car_number in range( 1, labels[1]+1 ):
        # Find pixels with each car_number label value
        nonzero = ( labels[0] == car_number ).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array( nonzero[0] )
        nonzerox = np.array( nonzero[1] )
        # Define a bounding box based on min/max x and y
        bbox = ( ( np.min( nonzerox ), np.min( nonzeroy ) ), ( np.max( nonzerox ), np.max( nonzeroy ) ) )
        # Draw the box on the image
        cv2.rectangle( img, bbox[0], bbox[1], (0,0,255), 6 )
    # Return the image
    return img

def draw_boxes( img, bboxes, color=( 0, 0, 255 ), thick = 6 ):
    """ Function to Draw boxes around identified cars """
    img_copy = np.copy( img )
    # Iterate over all discovered bounding boxes
    for box in bboxes:
        cv2.rectangle( img_copy, box[0], box[1], color, thick )
    return img_copy

# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel()
    # Return the feature vector
    return features

# Define a function to compute color histogram features 
# NEED TO CHANGE bins_range if reading .png files with mpimg!
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

#def get_hog_features( img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True ):
#    # Call with two outputs if vis==True
#    if vis == True:
#        features, hog_image = hog(img, 
#                                  orientations = orient, 
#                                  pixels_per_cell =( pix_per_cell, pix_per_cell ),
#                                  block_norm = 'L2-Hys',
#                                  cells_per_block = ( cell_per_block, cell_per_block ), 
#                                  transform_sqrt = True, 
#                                  visualise = vis, 
#                                  feature_vector = feature_vec )
#        return features, hog_image
#    # Otherwise call with one output
#    else:      
#        features = hog(img, 
#                       orientations = orient, 
#                       pixels_per_cell = ( pix_per_cell, pix_per_cell ),
#                       cells_per_block = ( cell_per_block, cell_per_block ), 
#                       block_norm = 'L2-Hys',
#                       transform_sqrt = True, 
#                       visualise = vis, 
#                       feature_vector = feature_vec )
#        return features
    
def get_hog_features(img, color_space='YUV'):
    # Call with two outputs if vis==True
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'Lab':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2Lab)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    hog = cv2.HOGDescriptor((64,64), (16,16), (8,8), (8,8), 9)

    return np.ravel(hog.compute(feature_image))
hog = cv2.HOGDescriptor((64,64), (8,8), (8,8), (8,8), 9)
image = mpimg.imread('test_images/test1.jpg')
hog.compute(image).shape

def extract_features( imgs, color_space='RGB', orient=9, pix_per_cell=8, cell_per_block=2, hog_channel=0 ):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        img = mpimg.imread( file )
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_img = cv2.cvtColor( img, cv2.COLOR_RGB2HSV )
            elif color_space == 'LUV':
                feature_img = cv2.cvtColor( img, cv2.COLOR_RGB2LUV )
            elif color_space == 'HLS':
                feature_img = cv2.cvtColor( img, cv2.COLOR_RGB2HLS )
            elif color_space == 'YUV':
                feature_img = cv2.cvtColor( img, cv2.COLOR_RGB2YUV )
            elif color_space == 'YCrCb':
                feature_img = cv2.cvtColor( img, cv2.COLOR_RGB2YCrCb )
        else: feature_img = np.copy( img )      

        ## Call get_hog_features() with vis=False, feature_vec=True
        #if hog_channel == 'ALL':
        #    hog_features = []
        #    for channel in range( feature_img.shape[2] ):
        #        hog_features.append(get_hog_features( feature_img[:,:,channel], 
        #                            orient, 
        #                            pix_per_cell, 
        #                            cell_per_block, 
        #                            vis = False, 
        #                            feature_vec = True ) )
        #    hog_features = np.ravel( hog_features )        
        #else:
        #    hog_features = get_hog_features(feature_img[:,:,hog_channel], orient, 
        #                                    pix_per_cell, 
        #                                    cell_per_block, 
        #                                    vis = False, 
        #                                    feature_vec = True )
        #features.append( hog_features )
        hog_features = np.ravel(hog.compute(feature_img))
        features.append(hog_features)
    return features
    #return np.concatenate( features )

## Get the Training data

In [ ]:
# Get all car images
car_images = glob.glob('vehicles/*/*.png')
print( "Number of car images = {}".format( len( car_images ) ) )

# Get all non-car images
non_car_images = glob.glob('non-vehicles/*/*.png')
print( "Number of non-car images = {}".format( len( non_car_images ) ) )

# Read random car image
car_path = car_images[ np.random.randint( 0, len( car_images ) ) ]
_, car_filename = os.path.split( car_path )
car_img = mpimg.imread( car_path )

# Read random non-car image
non_car_path = non_car_images[ np.random.randint( 0, len( non_car_images ) ) ]
_, non_car_filename = os.path.split( non_car_path )
non_car_img = mpimg.imread( non_car_path )

f, (ax1, ax2) = plt.subplots( 1, 2, figsize=( 25, 10 ) )
f.subplots_adjust( hspace = .2, wspace=.05 )
f.tight_layout()
ax1.imshow( car_img )
ax1.set_title( 'Car-' + car_filename, fontsize=40 )
ax2.imshow( non_car_img )
ax2.set_title( 'Non-Car-' + non_car_filename, fontsize=40 );       

## Get the Test Data

In [ ]:
# Get all car images
test_images = glob.glob( 'test_images/test*.jpg' )

print( "Number of test images = {}".format( len( test_images ) ) )

# Read a random test image
path = test_images[ np.random.randint( 0, len( test_images ) ) ]
_, filename = os.path.split( path )
img = mpimg.imread( path )

plt.imshow( img )
plt.title( filename, fontsize=40 );


## Visualize bounding boxes on the Test Data

In [ ]:
for path in sorted(test_images):
    _, filename = os.path.split( path )
    print("filename = ",filename)
    
    if filename == 'test1.jpg':
        bboxes = [ ( ( 810,400 ),( 950,500 ) ), ( ( 1045,400 ), ( 1270,505 ) ) ]
    elif filename == 'test2.jpg':
        bboxes = [ ( ( 0,0 ),( 0,0 ) ), ( ( 0,0 ), ( 0,0 ) ) ]
    elif filename == 'test3.jpg':
        bboxes = [ ( ( 865,410 ),( 970,470 ) ), ( ( 0,0 ), ( 0,0 ) ) ]
    elif filename == 'test4.jpg':
        bboxes = [ ( ( 810,400 ),( 950,495 ) ), ( ( 1030,400 ), ( 1260,505 ) ) ]
    elif filename == 'test5.jpg':
        bboxes = [ ( ( 810,400 ),( 950,495 ) ), ( ( 1075,400 ), ( 1280,505 ) ) ]
    elif filename == 'test6.jpg':
        bboxes = [ ( ( 810,400 ),( 950,495 ) ), ( ( 1005,400 ), ( 1205,505 ) ) ]
    else:
        print( "unknown filename" )
    img = mpimg.imread( path ) 
    _, filename = os.path.split( path )        
    box_img = draw_boxes( img, bboxes = bboxes )
    f, (ax1, ax2) = plt.subplots( 1, 2, figsize=( 25, 10 ) )
    f.subplots_adjust( hspace = .2, wspace=.05 )
    f.tight_layout()
    ax1.imshow( img )
    ax1.set_title( filename, fontsize=40 )
    ax2.imshow( box_img )
    ax2.set_title( 'Box-'+filename, fontsize=40 );


## Get Histogram of Gradient (HOG) and Visualize for Random image

In [ ]:
#
# Select a random car image and display its Histogram Of Gradients 
#
car_path = car_images[ np.random.randint( 0, len( car_images ) ) ]
_, car_filename = os.path.split( car_path )
car_img = mpimg.imread( car_path )
#_, hog_car = get_hog_features( car_img[:,:,2], 9, 8, 8, vis=True, feature_vec=True )
#_, hog_car = get_hog_features( car_img[:,:,2], 11, 16, 2, vis=True, feature_vec=True )
hog_car = get_hog_features( car_img, color_space=color_space )

f, (ax1, ax2) = plt.subplots( 1, 2, figsize=( 25, 10 ) )
f.subplots_adjust( hspace = .2, wspace=.05 )
f.tight_layout()
ax1.imshow( car_img )
ax1.set_title( 'Car-' + car_filename, fontsize=40 )
ax2.imshow( hog_car, cmap='gray' )
ax2.set_title( 'Hog-Car-' + car_filename, fontsize=40 );    

#
# Select a random non-car image and display its Histogram Of Gradients
#

non_car_path = non_car_images[ np.random.randint( 0, len( non_car_images ) ) ]
_, non_car_filename = os.path.split( non_car_path )
non_car_img = mpimg.imread( non_car_path )
#_, non_car_hog = get_hog_features( non_car_img[:,:,2], 9, 8, 8, vis=True, feature_vec=True )
#_, hog_car = get_hog_features( car_img[:,:,2], 11, 16, 2, vis=True, feature_vec=True )
hog_non_car = get_hog_features( non_car_img, color_space=color_space )

f, (ax1, ax2) = plt.subplots( 1, 2, figsize=( 25, 10 ) )
f.subplots_adjust( hspace = .2, wspace=.05 )
f.tight_layout()
ax1.imshow( non_car_img )
ax1.set_title( 'Non-Car-' + non_car_filename, fontsize=40 )
ax2.imshow( hog_non_car, cmap='gray' )
ax2.set_title( 'Hog-Non-Car-' + non_car_filename, fontsize=40 );    


## Extract all Features from both Data Sets

In [ ]:
car_features = extract_features(car_images,
                                color_space = color_space, 
                                orient = orient, 
                                pix_per_cell = pix_per_cell, 
                                cell_per_block = cell_per_block, 
                                hog_channel = hog_channel )

non_car_features = extract_features(non_car_images, 
                                    color_space = color_space, 
                                    orient = orient, 
                                    pix_per_cell = pix_per_cell, 
                                    cell_per_block = cell_per_block, 
                                    hog_channel = hog_channel )

print( 'Number of car_features = {}'.format( len( car_features[0] ) ) )

print( 'Number of non_car_features = {}'.format( len( non_car_features[0] ) ) )

## Normalize the data sets

In [ ]:
X = np.vstack( ( car_features, non_car_features ) ).astype( np.float64 )

# Fit a per-column scaler
X_scaler = StandardScaler().fit( X )

# Apply the scaler to X
scaled_X = X_scaler.transform( X )

# Define the labels vector
y = np.hstack( ( np.ones( len( car_features ) ), np.zeros( len( non_car_features ) ) ) )

## Visualize the normalized data sets

In [ ]:
#
# Display a random car image along with the un-normalized, and normalized image data
#
car_path = car_images[ np.random.randint( 0, len( car_images ) ) ]
_, car_filename = os.path.split( car_path )
car_img = mpimg.imread( car_path )

f, (ax1, ax2,ax3) = plt.subplots( 1, 3, figsize=( 25, 10 ) )
f.subplots_adjust( hspace = .2, wspace=.05 )
f.tight_layout()
ax1.imshow( car_img )
ax1.set_title( 'Car-' + car_filename, fontsize=40 )
ax2.plot( X[ 0 ])
ax2.set_title('Un-Normalized-' + car_filename, fontsize=30 )
ax3.plot( scaled_X[0] )
ax3.set_title( 'Normalized-' + car_filename, fontsize=30 );    

#
# Display a random non_car image along with the un-normalized, and normalized image data
#
non_car_path = non_car_images[ np.random.randint( 0, len( non_car_images ) ) ]
_, non_car_filename = os.path.split( non_car_path )
non_car_img = mpimg.imread( non_car_path )

f, (ax1, ax2,ax3) = plt.subplots( 1, 3, figsize=( 25, 10 ) )
f.subplots_adjust( hspace = .2, wspace=.05 )
f.tight_layout()
ax1.imshow( non_car_img )
ax1.set_title( 'Non-Car-' + non_car_filename, fontsize=30 )
ax2.plot( X[ 0 ])
ax2.set_title('Un-Normalized-' + non_car_filename, fontsize=30 )
ax3.plot( scaled_X[0] )
ax3.set_title( 'Normalized-' + non_car_filename, fontsize=30 );    

## Split up the data sets into training and test sets

In [ ]:
# split into 80% for training 20% for test
random_state = np.random.randint( 0, len( car_features ) )
X_train, X_test, y_train, y_test = train_test_split( scaled_X, y, test_size = 0.2, random_state=random_state )

## Train the model

In [ ]:
svc = LinearSVC()

svc.fit( X_train, y_train )

# Check the score of the SVC
print( 'LinearSVC Test Accuracy = {:.2%}'.format( svc.score( X_test, y_test ) ) )


## Save and Reload the model

In [ ]:
# save the mode
joblib.dump( ( svc, X_scaler ), 'model.pkl')

# reload
svc, X_scaler = joblib.load('model.pkl')

# Sliding Window Search

In [ ]:
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.75, 0.75)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_windows = np.int(xspan/nx_pix_per_step) 
    ny_windows = np.int(yspan/ny_pix_per_step)
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = (xs+1)*nx_pix_per_step + x_start_stop[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = (ys+1)*ny_pix_per_step + y_start_stop[0]
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

